- This file reads in the downloaded external data, formats them, and saves them in the ExternalDataFormatted Folder, 
- The DataPrepMergingFeatures_2 should be run after this

Imports

In [1]:
import pandas as pd
import numpy as np

Data Checklist
- New car registrations
- Public holidays
- Inflation
- GDP Growth
- USD TRY

### New car Registration Data

In [2]:
df = pd.read_excel("ExternalDataRaw\TUIKKaraTasıtYeniKayit.xls")
df.head()

,Year,Month,Akdeniz-TR6,Batı Anadolu-TR5,Batı Karadeniz-TR8,Batı Marmara-TR2,Doğu Karadeniz-TR9,Doğu Marmara-TR4,Ege-TR3,Güneydoğu Anadolu-TRC,İstanbul-TR1,Kuzeydoğu Anadolu-TRA,Orta Anadolu-TR7,Ortadoğu Anadolu-TRB
0,2013,01-Ocak,16090,13534,4292,4926,2631,10304,15519,7716,39681,1690,3969,2667
1,2013,02-Şubat,7566,6418,2049,2672,1111,4937,7820,3587,20793,834,1756,1137
2,2013,03-Mart,10920,9202,2885,3984,1524,6962,11546,4867,30890,1254,2652,1711
3,2013,04-Nisan,12895,9549,3236,4591,1684,8212,13697,5744,33105,1611,3113,1997
4,2013,05-Mayıs,16943,12884,4504,6444,2160,10850,18999,6993,48231,1948,4051,2485


- Combine date columns

In [3]:
df["Date"] = df["Year"].astype(str) + "-" + df['Month'].astype(str).str[0:2] + "-01"
df.drop(labels=["Month","Year"],axis=1,inplace=True)
df.head(1)

,Akdeniz-TR6,Batı Anadolu-TR5,Batı Karadeniz-TR8,Batı Marmara-TR2,Doğu Karadeniz-TR9,Doğu Marmara-TR4,Ege-TR3,Güneydoğu Anadolu-TRC,İstanbul-TR1,Kuzeydoğu Anadolu-TRA,Orta Anadolu-TR7,Ortadoğu Anadolu-TRB,Date
0,16090,13534,4292,4926,2631,10304,15519,7716,39681,1690,3969,2667,2013-01-01


In [4]:
df.columns

Index(['Akdeniz-TR6', 'Batı Anadolu-TR5', 'Batı Karadeniz-TR8',
       'Batı Marmara-TR2', 'Doğu Karadeniz-TR9', 'Doğu Marmara-TR4', 'Ege-TR3',
       'Güneydoğu Anadolu-TRC', 'İstanbul-TR1', 'Kuzeydoğu Anadolu-TRA',
       'Orta Anadolu-TR7', 'Ortadoğu Anadolu-TRB', 'Date'],
      dtype='object')

Combine into following regions
- Marmara Bölgesi    
    - TR2,TR4,TR1 
- Ege Bölgesi  
    - TR3            
- İç Anadolu Bölgesi 
    - TR5,TR7       
- Akdeniz Bölgesi
    - TR6               
- Karadeniz Bölgesi    
    - TR8,TR9        
- Güneydoğu Anadolu Bölgesi  
    - TRC   
- Doğu Anadolu Bölgesi    
    - TRA,TRB      

Create new columns with the sum of each region

In [5]:
df["Marmara Bölgesi"] = df[['Batı Marmara-TR2','Doğu Marmara-TR4','İstanbul-TR1']].sum(axis=1)
df["Ege Bölgesi"] = df['Ege-TR3']
df["İç Anadolu Bölgesi"] = df[['Batı Anadolu-TR5', 'Orta Anadolu-TR7']].sum(axis=1)
df["Akdeniz Bölgesi"] = df['Akdeniz-TR6']
df["Karadeniz Bölgesi"] = df[['Doğu Karadeniz-TR9','Batı Karadeniz-TR8']].sum(axis=1)
df["Güneydoğu Anadolu Bölgesi"] =  df['Güneydoğu Anadolu-TRC']
df["Doğu Anadolu Bölgesi"] = df[['Kuzeydoğu Anadolu-TRA','Ortadoğu Anadolu-TRB']].sum(axis=1)

Drop old columns

In [7]:
df = df[["Date","Akdeniz Bölgesi", "İç Anadolu Bölgesi","Karadeniz Bölgesi","Marmara Bölgesi", "Ege Bölgesi", "Güneydoğu Anadolu Bölgesi",
"Doğu Anadolu Bölgesi"]]
df.head(1)

,Date,Akdeniz Bölgesi,İç Anadolu Bölgesi,Karadeniz Bölgesi,Marmara Bölgesi,Ege Bölgesi,Güneydoğu Anadolu Bölgesi,Doğu Anadolu Bölgesi
0,2013-01-01,16090,17503,6923,54911,15519,7716,4357



Convert to datetime object and add new column indicating when the data is available (t+2 months)

In [8]:
df["Date"]= pd.to_datetime(df['Date'])
df["DataReleaseDate"] = df["Date"] + pd.to_timedelta(50,unit='d')
df.head(1)

,Date,Akdeniz Bölgesi,İç Anadolu Bölgesi,Karadeniz Bölgesi,Marmara Bölgesi,Ege Bölgesi,Güneydoğu Anadolu Bölgesi,Doğu Anadolu Bölgesi,DataReleaseDate
0,2013-01-01,16090,17503,6923,54911,15519,7716,4357,2013-02-20


Set the data release date to nearest sunday

In [9]:
df["DataReleaseDate"] = df["DataReleaseDate"] - pd.to_timedelta(df["DataReleaseDate"].dt.dayofweek, unit='d') +  pd.to_timedelta(6, unit='d')

Pivot table so its easier to merge

In [10]:
df = df.melt(id_vars=["Date", "DataReleaseDate"],
var_name="Region",
value_name="Cars")

Filter out early dates

In [11]:
df = df[df["DataReleaseDate"] > "2013-12-01" ]
df.sort_values(by=["Region","Date"],inplace=True)
df.reset_index(inplace=True,drop=True)

In [12]:
df.head()

,Date,DataReleaseDate,Region,Cars
0,2013-11-01,2013-12-22,Akdeniz Bölgesi,12007
1,2013-12-01,2014-01-26,Akdeniz Bölgesi,9999
2,2014-01-01,2014-02-23,Akdeniz Bölgesi,15874
3,2014-02-01,2014-03-23,Akdeniz Bölgesi,6537
4,2014-03-01,2014-04-20,Akdeniz Bölgesi,8564


Now want to transform this into a 12m cumulative sum 

In [13]:
regions = df["Region"].unique()
for region in regions:
    df.loc[df["Region"]==region,"Cars(12m)"] = df[df.Region == region]["Cars"].rolling(12).sum()

Drop monthly car column

In [14]:
df.drop(labels="Cars",axis=1,inplace=True)
df.rename(columns={"Cars(12m)":"Cars"},inplace=True)

- Put these into a weekly df

In [15]:
df_dates = pd.date_range('2014-01-05', '2019-12-30', freq='1W').to_frame(name='Date').reset_index(drop=True)
df_dates = pd.DataFrame(np.repeat(df_dates.values, 7, axis=0))
df_dates.columns = ["Date"]
df_dates["Region"] = ["Akdeniz Bölgesi", "İç Anadolu Bölgesi","Karadeniz Bölgesi","Marmara Bölgesi", "Ege Bölgesi", "Güneydoğu Anadolu Bölgesi",
"Doğu Anadolu Bölgesi"]*313

In [16]:
df_dates.sort_values(by=["Region","Date"],inplace=True)
df_dates.reset_index(inplace=True,drop=True)

Below is an inefficient loop to properly set up the data release dates

In [17]:
df_dates["Cars"] = 0

for i in range(len(df_dates)):
    for k in range(len(df)-1):
        if df_dates.loc[i,"Region"] == df.loc[k,"Region"]:
            if df_dates.loc[i,"Date"] > df.loc[k,"DataReleaseDate"] and df_dates.loc[i,"Date"] <= df.loc[k+1,"DataReleaseDate"]:
                #print(df_dates.loc[i,"Date"], 
                #df.loc[k,"DataReleaseDate"],
                #df.loc[k+1,"DataReleaseDate"])
                df_dates.loc[i,"Cars"] = df.loc[k,"Cars"]

Add one day so it matches with the training/test data (Sundays)

In [18]:
df_dates["Date"] = df_dates["Date"] + pd.to_timedelta(1,unit='d')
df_dates[100:]

,Date,Region,Cars
100,2015-12-07,Akdeniz Bölgesi,150190.0
101,2015-12-14,Akdeniz Bölgesi,150190.0
102,2015-12-21,Akdeniz Bölgesi,150190.0
103,2015-12-28,Akdeniz Bölgesi,150190.0
104,2016-01-04,Akdeniz Bölgesi,150325.0
...,...,...,...
2186,2019-12-02,İç Anadolu Bölgesi,90466.0
2187,2019-12-09,İç Anadolu Bölgesi,90466.0
2188,2019-12-16,İç Anadolu Bölgesi,90466.0
2189,2019-12-23,İç Anadolu Bölgesi,90466.0


In [19]:
df_dates.reset_index(inplace=True,drop=True)
df_dates.to_csv("ExternalDataFormatted\CarRegistrationDataReady.csv",index=False)

## Public Holiday Data

In [20]:
df = pd.read_excel("ExternalDataRaw\Holidays.xlsx")
df.head()

,Date,Holiday,Official,Religious,Weekend,Weekday
0,2014-01-01,New years,1,0,0,1
1,2014-04-23,23rd April,1,0,0,1
2,2014-05-01,Workers,1,0,0,1
3,2014-05-19,Youth and Sports,1,0,0,1
4,2014-07-27,Ramadan,0,1,1,0


In [21]:
df.drop(labels=["Holiday","Official", "Religious"],axis=1, inplace=True)

Round holiday dates to week start dates

In [22]:
df["Date"]  = df["Date"] - pd.to_timedelta(df["Date"].dt.dayofweek, unit='d')

Sum up all holidays for the given week

In [23]:
df= df.groupby(df['Date']).aggregate("sum")
df.reset_index(inplace=True)
df.head()

,Date,Weekend,Weekday
0,2013-12-30,0,1
1,2014-04-21,0,1
2,2014-04-28,0,1
3,2014-05-19,0,1
4,2014-07-21,1,0


In [24]:
df.columns=["Date", "WeekendHolidayCount", "WeekDayHolidayCount"]

In [25]:
df.to_csv("ExternalDataFormatted\HolidayDataReady.csv", index=False)

### Inflation data

In [26]:
df = pd.read_excel("ExternalDataRaw\Inflation.xls")
df.head()

,Year,Month,MoMInflation%
0,2013,01-Ocak,1.65
1,2013,02-Şubat,1.95
2,2013,03-Mart,2.63
3,2013,04-Nisan,3.06
4,2013,05-Mayıs,3.21


Configure Data column

In [27]:
df["Date"] = df["Year"].astype(str) + "-" + df['Month'].astype(str).str[0:2] + "-01"
df.drop(labels=["Month","Year"],axis=1,inplace=True)

Convert to datetime object and add new column indicating when the data is available (t+1 months)

In [29]:
df["Date"]= pd.to_datetime(df['Date'])
df["DataReleaseDate"] = df["Date"] + pd.to_timedelta(32,unit='d')

In [30]:
# Set the data release date to nearest sunday
df["DataReleaseDate"] = df["DataReleaseDate"] - pd.to_timedelta(df["DataReleaseDate"].dt.dayofweek, unit='d') +  pd.to_timedelta(6, unit='d')

In [31]:
df.head()

,MoMInflation%,Date,DataReleaseDate
0,1.65,2013-01-01,2013-02-03
1,1.95,2013-02-01,2013-03-10
2,2.63,2013-03-01,2013-04-07
3,3.06,2013-04-01,2013-05-05
4,3.21,2013-05-01,2013-06-02


- Put these into a weekly df

In [32]:
df_dates = pd.date_range('2014-01-05', '2019-12-30', freq='1W').to_frame(name='Date').reset_index(drop=True)
df_dates.columns = ["Date"]
df_dates.sort_values(by="Date",inplace=True)
df_dates.reset_index(inplace=True,drop=True)

Again an inefficient loop to configure the data release dates

In [33]:
df_dates["MoMInflation%"] = 0

for i in range(len(df_dates)):
    for k in range(len(df)-1):
        if df_dates.loc[i,"Date"] > df.loc[k,"DataReleaseDate"] and df_dates.loc[i,"Date"] <= df.loc[k+1,"DataReleaseDate"]:
            df_dates.loc[i,"MoMInflation%"] = df.loc[k,"MoMInflation%"]

Add one day so it matches with the training/test data (Sundays)

In [34]:
df_dates["Date"] = df_dates["Date"] + pd.to_timedelta(1,unit='d')

In [35]:
df_dates.columns = ["DateDataAvailable", "InflationYTD"]

In [36]:
df_dates.head()

,DateDataAvailable,InflationYTD
0,2014-01-06,6.91
1,2014-01-13,7.40
2,2014-01-20,7.40
3,2014-01-27,7.40
4,2014-02-03,7.40


In [37]:
df_dates.reset_index(inplace=True,drop=True)
df_dates.to_csv("ExternalDataFormatted\InflationDataReady.csv",index=False)

# GDP Growth Data

In [38]:
df = pd.read_excel("ExternalDataRaw\GDPGrowth.xlsx")
df.columns = ["DataReleaseDate", "GDPGrowth%"]
df.head()

,DataReleaseDate,GDPGrowth%
0,"Dec 02, 2019",0.009
1,"Sep 02, 2019",-0.015
2,"May 31, 2019",-0.026
3,"Mar 11, 2019",-0.030
4,"Dec 10, 2018",0.016


In [39]:
df["DataReleaseDate"] = pd.to_datetime(df['DataReleaseDate'])
df.sort_values(by="DataReleaseDate",inplace=True)
df.reset_index(inplace=True,drop=True)
df.head(-1)

,DataReleaseDate,GDPGrowth%
0,2013-12-10,0.044
1,2014-03-31,0.044
2,2014-06-10,0.043
3,2014-09-10,0.021
4,2014-12-10,0.017
5,2015-03-31,0.026
6,2015-06-10,0.023
7,2015-09-10,0.038
8,2015-12-10,0.040
9,2016-03-31,0.057


- Put these into a weekly df

In [40]:
df_dates = pd.date_range('2014-01-05', '2019-12-30', freq='1W').to_frame(name='Date').reset_index(drop=True)
df_dates.columns = ["Date"]
df_dates.sort_values(by="Date",inplace=True)
df_dates.reset_index(inplace=True,drop=True)

Again the ugly loop :/

In [41]:
df_dates["GDPGrowth%"] = 0

for i in range(len(df_dates)):
    for k in range(len(df)-1):
        if df_dates.loc[i,"Date"] > df.loc[k,"DataReleaseDate"] and df_dates.loc[i,"Date"] <= df.loc[k+1,"DataReleaseDate"]:
            df_dates.loc[i,"GDPGrowth%"] = df.loc[k,"GDPGrowth%"]

    # set last 4 entries manually
    if i >= 309:
        df_dates.loc[i,"GDPGrowth%"] = df.loc[len(df)-1,"GDPGrowth%"]

Add one day so it matches with the training/test data

In [43]:
df_dates["Date"] = df_dates["Date"] + pd.to_timedelta(1,unit='d')
df_dates

,Date,GDPGrowth%
0,2014-01-06,0.044
1,2014-01-13,0.044
2,2014-01-20,0.044
3,2014-01-27,0.044
4,2014-02-03,0.044
...,...,...
308,2019-12-02,-0.015
309,2019-12-09,0.009
310,2019-12-16,0.009
311,2019-12-23,0.009


In [44]:
df_dates.reset_index(inplace=True,drop=True)
df_dates.to_csv("ExternalDataFormatted\GDPDataReady.csv",index=False)

# USD TRY DATA

In [45]:
df = pd.read_csv(r"ExternalDataRaw\USDTRYWeeklyChanges.csv")
df.head()

,Date,WoWChange%
0,29.12.2019,"0,37%"
1,22.12.2019,"0,23%"
2,15.12.2019,"2,21%"
3,08.12.2019,"0,55%"
4,01.12.2019,"0,52%"


In [46]:
df["Date"] = pd.to_datetime(df['Date'],format="%d.%m.%Y")
df.sort_values(by="Date",inplace=True)
df.reset_index(inplace=True,drop=True)
df["Date"] = df["Date"] +  pd.to_timedelta(1,unit='d')
df.head()

,Date,WoWChange%
0,2014-01-06,-0.62%
1,2014-01-13,3.08%
2,2014-01-20,4.47%
3,2014-01-27,"-3,04%"
4,2014-02-03,"-1,80%"


In [47]:
df.to_csv(r"ExternalDataFormatted\USDDataReady.csv",index=False)